In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy.io
import glob
import os
from math import sqrt
import json
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import pathlib
from PIL import Image

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
batch_size = 16
lr = 0.001
epochs = 50

In [ ]:
training_dir = 'training_vacancy/'
testing_dir = 'testing_vacancy/'
train_count=len(glob.glob(training_dir+'/**/*.jpg'))
test_count=len(glob.glob(testing_dir+'/**/*.jpg'))
print('train : {}, test : {}'.format(train_count,test_count))

In [ ]:
transformer = transforms.Compose([
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [ ]:
train_loader = DataLoader(
    torchvision.datasets.ImageFolder(training_dir,transform = transformer),
    batch_size = 512,
    shuffle = True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(testing_dir,transform=transformer),
    batch_size=32, shuffle=True
)

In [ ]:
classes = ['vacant','non-vacant','parking']

In [ ]:
class VPSNet(nn.Module):
    def __init__(self,num_classes = 3):
        super(VPSNet,self).__init__()
        
        self.model = nn.Sequential(
            nn.Conv2d(3, 40, kernel_size=(3, 9), stride=(1, 2)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(3, 3), stride=2),
            nn.Conv2d(40, 80, kernel_size=(3, 5), padding=(1, 0)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=(1, 0)),
            nn.Conv2d(80, 120, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(120, 160, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Flatten(),
            nn.Dropout(),
            nn.Linear(160 * 5 * 5, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes)
        )
    def forward(self,x):
        x = self.model(x)
        return x

In [ ]:
model = VPSNet(num_classes=3).to(device)

In [ ]:
optimizer = Adam(model.parameters(),lr = lr,betas=(0.9,0.999),eps = 1e-08,weight_decay=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(epochs):
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    
    for i,(images,labels) in enumerate(train_loader):
        
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        optimizer.zero_grad()
        loss = loss_fn
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.cpu().data*images.size(0)
        _,prediction = torch.max(outputs.data,1)
        train_accuracy += int(torch.sum(prediction == labels.data))
    train_accuracy = train_accuracy/train_count
    train_loss = train_loss/train_count
    
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        print(images.shape)
        images=Variable(images.cuda())
        labels=Variable(labels.cuda())  
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    print('epoch : {} , train loss: {} , train accuracy : {}, test accuracy : {}'.format(epoch,train_loss,train_accuracy,test_accuracy))

In [ ]:
#save or load model
#save
path = 'model_save/98_test.pth'
#path = 'model_save/98_test.pt'
#torch.save(model.state_dict(),path)
#model.save(path)
#load
model = VPSNet()
model.load_state_dict(torch.load(path))
model.cuda()
#model = torch.load(path)

In [ ]:
def image_loader(path):
    loader = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
    image = Image.open(path)
    image = loader(image).float()
    image = Variable(image)
    image = image.unsqueeze(0)
    return image.cuda()

def pred_image(model,img_name):
    img = image_loader(img_name)
    prediction = model(img)
    prediction = torch.argmax(prediction.data).cpu().item()
    classification = 'vacant'
    if (prediction == 2):
        classification = 'vacant'
    elif (prediction == 0):
        classification = 'non-vacant'
    elif (prediction == 1):
        classification = 'parking'
    return classification

In [ ]:
#visualize output
model.eval()
sample_id = 177
testing_dir = 'testing_vacancy/'
test_class = 'parking/'
for file in glob.glob(testing_dir+test_class+'/*.jpg'):
    classification = pred_image(model,file)
    print(classification)